In [56]:
import csv
# Specify the path to your CSV file
serve_file_path = '/Users/noahjillson/Desktop/MCM/top_100_serving_player_stats.csv'
return_file_path = '/Users/noahjillson/Desktop/MCM/top_100_returning_player_stats.csv'
wimbledon_file_path = '/Users/noahjillson/Desktop/MCM/wimbledon_player_stats.csv'
matches_file_path = '/Users/noahjillson/Desktop/MCM/Wimbledon_featured_matches.csv'

# Top 100 server statistics
serve_stats = []
with open(serve_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        serve_stats.append(row) # Each row is a dictionary with keys as header names

# Top 100 returning statistics
return_stats = []
with open(return_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        return_stats.append(row) # Each row is a dictionary with keys as header names

# Wimbledon 2023 player statistics
wimbledon_stats = []
with open(wimbledon_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        wimbledon_stats.append(row) # Each row is a dictionary with keys as header names

all_points = []
with open(matches_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        all_points.append(row) # Each row is a dictionary with keys as header names

In [39]:
wimbledon_stats

[{'player_name': 'Daniil Medvedev',
  '1st_serve_in_percent': '61',
  '1st_serve_won_percent': '75',
  '2nd_serve_won_percent': '52',
  '1st_return_won_percent': '31',
  '2nd_return_won_percent': '54'},
 {'player_name': 'Carlos Alcaraz',
  '1st_serve_in_percent': '65',
  '1st_serve_won_percent': '71',
  '2nd_serve_won_percent': '55',
  '1st_return_won_percent': '34',
  '2nd_return_won_percent': '53'},
 {'player_name': 'Nicolas Jarry',
  '1st_serve_in_percent': '64',
  '1st_serve_won_percent': '74',
  '2nd_serve_won_percent': '53',
  '1st_return_won_percent': '26',
  '2nd_return_won_percent': '46'},
 {'player_name': 'Lorenzo Musetti',
  '1st_serve_in_percent': '64',
  '1st_serve_won_percent': '69',
  '2nd_serve_won_percent': '51',
  '1st_return_won_percent': '30',
  '2nd_return_won_percent': '51'},
 {'player_name': 'Hubert Hurkacz',
  '1st_serve_in_percent': '64',
  '1st_serve_won_percent': '76',
  '2nd_serve_won_percent': '52',
  '1st_return_won_percent': '27',
  '2nd_return_won_percen

## Calculating $f_{\text{avg}}$ and $g_{\text{avg}}$

In [32]:
# Serving Averages
a_avg = sum(map(lambda x: float(x['1st_serve_in_percent']), serve_stats)) / 10000   # We divide by 10000 to acount for 100 entries
b_avg = sum(map(lambda x: float(x['1st_serve_won_percent']), serve_stats)) / 10000  # and for the conversion of the percentages to
c_avg = sum(map(lambda x: float(x['2nd_serve_won_percent']), serve_stats)) / 10000  # decimal values (i.e. 79.3 -> 0.793)

# Returning Averages
d_avg = sum(map(lambda x: float(x['1st_serve_return_won']), return_stats)) / 10000
e_avg = sum(map(lambda x: float(x['2nd_serve_return_won']), return_stats)) / 10000

print(a_avg, b_avg, c_avg, d_avg, e_avg)

0.6267299999999998 0.7444099999999999 0.5236600000000002 0.26203000000000004 0.48307


In [34]:
f_avg = a_avg * b_avg + (1 - a_avg) * c_avg
g_avg = a_avg * d_avg + (1 - a_avg) * e_avg

print(f_avg, g_avg, f_avg + g_avg)

0.6620106475 0.34453760080000007 1.0065482483000001


## Defining functions for $f_i$ and $g_i$

In [35]:
def f_i(a_i, b_i, c_i):
    return a_i * b_i + (1 - a_i) * c_i

def g_i(d_i, e_i):
    return a_avg * d_i + (1 - a_avg) * e_i

## Calculating $f_i$ and $g_i$ for Wimbledon Players

In [66]:
# Cleaning the Wimbledon player's statistics
for p in wimbledon_stats:
    p['1st_serve_in_percent'] = float(p['1st_serve_in_percent']) / 100
    p['1st_serve_won_percent'] = float(p['1st_serve_won_percent']) / 100
    p['2nd_serve_won_percent'] = float(p['2nd_serve_won_percent']) / 100
    p['2nd_return_won_percent'] = float(p['2nd_return_won_percent']) / 100
    p['1st_return_won_percent'] = float(p['1st_return_won_percent']) / 100

In [67]:
for p in wimbledon_stats:
    p['f_i'] = f_i(p['1st_serve_in_percent'], p['1st_serve_won_percent'], p['2nd_serve_won_percent'])
    p['g_i'] = g_i(p['1st_return_won_percent'], p['2nd_return_won_percent'])


In [49]:
# out_path = '/Users/noahjillson/Desktop/MCM/computed_Wimbledon_player_stats.csv'

# fieldnames = ['player_name','1st_serve_in_percent','1st_serve_won_percent','2nd_serve_won_percent','1st_return_won_percent','2nd_return_won_percent','g_i','f_i']
# with open(out_path, mode='w', newline='') as file:
#     writer = csv.DictWriter(file, fieldnames=fieldnames)
#     writer.writeheader()
#     writer.writerows(wimbledon_stats)

## Defining $f_{i,j}$ and $g_{i,j}$

In [69]:
def f_ij(fi, gj):
    return f_avg + (fi - f_avg) - (gj - g_avg)

def g_ji(fi, gj):
    return g_avg + (gj - g_avg) - (fi - f_avg)

## Calculating $f_{i,j}$ and $g_{i,j}$ for all matches

In [60]:
matches = set()
for r in all_points:
    matches.add((r['player1'], r['player2']))

In [76]:
def find(name):
    for r in wimbledon_stats:
        if r['player_name'] == name:
            return r
    print(f"Could not find {name} in wimbledon_stats")

p_dict = {}
for m in matches:
    fij = f_ij(find(m[0])['f_i'], find(m[1])['g_i'])
    gji = g_ji(find(m[0])['f_i'], find(m[1])['g_i'])
    p_dict[m] = [fij, f_ij(find(m[1])['f_i'], find(m[0])['g_i'])]
    #print(m, fij, f_ij(find(m[1])['f_i'], find(m[0])['g_i']))

for prob in p_dict:
    print(prob, p_dict[prob])

('Carlos Alcaraz', 'Nicolas Jarry') [0.6638836008, 0.5980163008]
('Alexander Zverev', 'Matteo Berrettini') [0.6557490008, 0.6536509008]
('Carlos Alcaraz', 'Novak Djokovic') [0.5801509007999999, 0.6106163008000001]
('Stan Wawrinka', 'Novak Djokovic') [0.5821509008, 0.6506163008000001]
('Laslo Djere', 'Stefanos Tsitsipas') [0.6058836008, 0.6600836008000001]
('Carlos Alcaraz', 'Holger Rune') [0.5965566008, 0.5833163008]
('Daniel Elahi Galan', 'Mikael Ymer') [0.5696163008, 0.5686836008]
('Jannik Sinner', 'Roman Safiullin') [0.6307855008000001, 0.6047509008]
('Andrey Rublev', 'Novak Djokovic') [0.5825509008, 0.6356855008000001]
('Jiri Lehecka', 'Tommy Paul') [0.5977509007999999, 0.6295163008]
('Jannik Sinner', 'Novak Djokovic') [0.5782509008, 0.6231509008]
('Hubert Hurkacz', 'Novak Djokovic') [0.5997509007999999, 0.6731509008000001]
('Lorenzo Musetti', 'Hubert Hurkacz') [0.6213509008, 0.6397509008]
('Christopher Eubanks', 'Stefanos Tsitsipas') [0.6462836008, 0.7163509008]
('Andrey Rublev', 